In [ ]:
from binance.um_futures import UMFutures
from binance.spot import Spot
from binance.client import Client
import time
import numpy as np
import pandas as pd
import matplotlib as plot
from datetime import datetime, timedelta
import math
import matplotlib.pyplot as plt
from matplotlib import cm
import csv

#### Binance期貨資料下載

In [ ]:
all_symbols = ['LINKUSDT','FTMUSDT','RUNEUSDT','UNIUSDT','GRTUSDT',
               'RSRUSDT','TRBUSDT','MKBUSDT','AAVEUSDT','THETAUSDT',
               'CRVUSDT','RAYUSDT','ZRXUSDT','SNXUSDT','XTZUSDT',]

for ff in all_symbols:
    # config_logging(logging, logging.DEBUG)
    Futures_client = UMFutures(timeout = 5)

    start_time_date = datetime(2022,1,1)
    end_time_date = datetime(2024,5,10)
    klines_raw = []

    while start_time_date < end_time_date:
        start_time_ts = datetime.timestamp(start_time_date)
        klines_raw = klines_raw + Futures_client.klines(symbol = ff, interval = "4h", startTime = int(start_time_ts)*1000, limit = 500)
        start_time_date = start_time_date + timedelta(hours=4*500)
        time.sleep(30)

    # preprocessing 1
    headers = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_vol', '#trade', 'taker_buy_vol', 'taker_buy_quote_vol', 'ignore']
    klines_df = pd.DataFrame(klines_raw, columns = headers, dtype=float)
    klines_df.drop(columns=['ignore', 'quote_vol', 'taker_buy_quote_vol'], inplace=True)

    # Convert timestamps to datetime objects
    klines_df['open_time'] = list(map(lambda x: datetime.fromtimestamp(x/1000),klines_df['open_time']))
    klines_df['close_time'] = list(map(lambda x: datetime.fromtimestamp(x/1000),klines_df['close_time']))
    # preprocessing 4
    first_day_of_week = 3 # the day of the week of the very first day of the interval
    week_list = list((klines_df.index + first_day_of_week * 96) // 96 % 7)
    klines_df['week'] = week_list

    # Save dataframe to CSV file
    klines_df.to_csv(ff + '_期貨.csv', index=False)


#### Binance現貨資料下載

In [ ]:
all_symbols = ['LINKUSDT','FTMUSDT','RUNEUSDT','UNIUSDT','GRTUSDT',
               'RSRUSDT','TRBUSDT','MKBUSDT','AAVEUSDT','THETAUSDT',
               'CRVUSDT','RAYUSDT','ZRXUSDT','SNXUSDT','XTZUSDT',]

for ss in all_symbols:
    # config_logging(logging, logging.DEBUG)
    Spot_client = Spot(timeout = 5)

    start_time_date = datetime(2022,1,1)
    end_time_date = datetime(2024,5,10)
    klines_raw = []

    while start_time_date < end_time_date:
        start_time_ts = datetime.timestamp(start_time_date)
        klines_raw = klines_raw + Spot_client.klines(symbol = ss, interval = "4h", startTime = int(start_time_ts)*1000, limit = 500)
        start_time_date = start_time_date + timedelta(hours=4*500)
        time.sleep(30)

    # preprocessing 1
    headers = ['open_time', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_vol', '#trade', 'taker_buy_vol', 'taker_buy_quote_vol', 'ignore']
    klines_df = pd.DataFrame(klines_raw, columns = headers, dtype=float)
    klines_df.drop(columns=['ignore', 'quote_vol', 'taker_buy_quote_vol'], inplace=True)

    # Convert timestamps to datetime objects
    klines_df['open_time'] = list(map(lambda x: datetime.fromtimestamp(x/1000),klines_df['open_time']))
    klines_df['close_time'] = list(map(lambda x: datetime.fromtimestamp(x/1000),klines_df['close_time']))
    # preprocessing 4
    first_day_of_week = 3 # the day of the week of the very first day of the interval
    week_list = list((klines_df.index + first_day_of_week * 96) // 96 % 7)
    klines_df['week'] = week_list

    # Save dataframe to CSV file
    klines_df.to_csv(ss + '_現貨.csv', index=False)

#### Binance 資金費率下載

In [ ]:
# 填入你的 Binance API key 和 secret
api_key = 'SOvKFyvNT6rC6OZvJ569D8LxUCWW4PVQOliJKfZ0mfRC0R85W6KZVQnAEgzNWL9e'
api_secret = 'OvtqqZCslAbfQvmt8SjrgASaJawebaNYa7sIgPBkFiBNCaiTS5RdAzXW7jR9OMnL'

client = Client(api_key, api_secret)

# 定義時間區段
start_str = '2022-01-01'
end_str = '2024-05-17'
start_date = datetime.strptime(start_str, '%Y-%m-%d')
end_date = datetime.strptime(end_str, '%Y-%m-%d')

# 定義所需要的幣種
symbols = ['LINKUSDT','FTMUSDT','RUNEUSDT','UNIUSDT','GRTUSDT',
               'RSRUSDT','TRBUSDT','MKBUSDT','AAVEUSDT','THETAUSDT',
               'CRVUSDT','RAYUSDT','ZRXUSDT','SNXUSDT','XTZUSDT',]

# 定義每次請求所需要的時間段(避免一次要求太多)
time_interval = timedelta(days=15)

# 獲取數據並存到csv檔案裡面
for symbol in symbols:
    funding_rates = []
    current_start = start_date
    while current_start < end_date:
        current_end = min(current_start + time_interval, end_date)
        funding_rate_data = client.futures_funding_rate(
            symbol=symbol,
            startTime=int(current_start.timestamp() * 1000),
            endTime=int(current_end.timestamp() * 1000)
        )
        funding_rates.extend(funding_rate_data)
        current_start = current_end
    
    # 保存數據到各自的csv檔案
    csv_file = f'{symbol}_funding_rates.csv'
    with open(csv_file, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['open_time', 'fundingRate'])
        for rate in funding_rates:
            # 轉換時間為UTC+0
            funding_time_utc = datetime.utcfromtimestamp(rate['fundingTime'] / 1000).strftime('%Y/%m/%d %H:%M')
            writer.writerow([funding_time_utc, rate['fundingRate']])
    
    print(f"{symbol} 資金費率數據已保存到 {csv_file}")